<a href="https://colab.research.google.com/github/jelittle/4413-final-project/blob/colour-simulation/CarColourSimulation/ultralytics_yolo_car_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset Structure

The data distribution within the Carparts Segmentation Dataset is organized as outlined below:

- **Training set**: Includes 3156 images, each accompanied by its corresponding annotations.
- **Testing set**: Comprises 276 images, with each one paired with its respective annotations.
- **Validation set**: Consists of 401 images, each having corresponding annotations.

## Setup

pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://www.pepy.tech/projects/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [5]:
!pip install ultralytics
!pip install --upgrade dataset-tools
!pip install supervisely

In [2]:

import ultralytics
ultralytics.checks()

Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 42.6/112.6 GB disk)


download dataset and convert dataset to yolo format



In [4]:
import dataset_tools as dtools

dtools.download(dataset='Car Segmentation', dst_dir='~/dataset-ninja/')






Unpacking 'car-segmentation.tar': 100%|██████████| 429/429 [00:01<00:00, 360.58file/s] 


'/root/dataset-ninja/car-segmentation'

In [ ]:
import supervisely as sly

In [8]:
import supervisely as sly
import os

dataset_path = '/root/dataset-ninja/car-segmentation'  # Replace with the actual path to your Supervisely dataset
yolo_dir = '/root/yolo_dataset'  # Replace with the desired output directory

# Ensure the output directory exists
os.makedirs(yolo_dir, exist_ok=True)

# Convert the dataset to YOLO format
sly.convert.to_yolo(dataset_path, yolo_dir)

print(f"Dataset converted and saved to: {yolo_dir}")

Converting Supervisely project to YOLO format: 100%|██████████| 211/211 [00:03<00:00, 57.72it/s]
{"message": "Dataset 'ds' has been converted to YOLO format.", "timestamp": "2025-06-30T01:31:06.707Z", "level": "info"}
{"message": "Project 'car-segmentation' has been converted to YOLO format.", "timestamp": "2025-06-30T01:31:06.708Z", "level": "info"}


Dataset converted and saved to: /root/yolo_dataset


## Train

Train YOLO11 on [Detect](https://docs.ultralytics.com/tasks/detect/), [Segment](https://docs.ultralytics.com/tasks/segment/), [Classify](https://docs.ultralytics.com/tasks/classify/) and [Pose](https://docs.ultralytics.com/tasks/pose/) datasets. See [YOLO11 Train Docs](https://docs.ultralytics.com/modes/train/) for more information.

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n-seg.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="carparts-seg.yaml", epochs=10, imgsz=640, batch=32, workers=64)

![Dataset sample image](https://github.com/ultralytics/docs/releases/download/0/dataset-sample-image.avif)

## Predict

YOLO11 may be used directly in the Command Line Interface (CLI) with a yolo command for a variety of tasks and modes and accepts additional arguments, i.e. imgsz=640. See a full list of available [yolo arguments](https://docs.ultralytics.com/usage/cfg/) and other details in the [YOLO11 Predict Docs](https://docs.ultralytics.com/modes/train/).

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO(f"{model.trainer.save_dir}/weights/best.pt")  # load a fine-tuned model

# Inference using the model (img/video/stream)
prediction_results = model.predict("https://github.com/ultralytics/assets/releases/download/v0.0.0/carparts-image.jpg", save=True)

print(prediction_results)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img align="left" src="https://github.com/user-attachments/assets/436ded05-9203-4bb7-883b-f7a1f9a399c1" width="600">

## Export

Export a YOLO11 model to any supported format below with the `format` argument, i.e. `format=onnx`. See [YOLO11 Export Docs](https://docs.ultralytics.com/modes/export/) for more information.

- 💡 ProTip: Export to [ONNX](https://docs.ultralytics.com/integrations/onnx/) or [OpenVINO](https://docs.ultralytics.com/integrations/openvino/) for up to 3x CPU speedup.  
- 💡 ProTip: Export to [TensorRT](https://docs.ultralytics.com/integrations/tensorrt/) for up to 5x GPU speedup.

| Format                                                                   | `format` Argument | Model                     | Metadata | Arguments                                                            |
|--------------------------------------------------------------------------|-------------------|---------------------------|----------|----------------------------------------------------------------------|
| [PyTorch](https://pytorch.org/)                                          | -                 | `yolo11n.pt`              | ✅        | -                                                                    |
| [TorchScript](https://docs.ultralytics.com/integrations/torchscript)     | `torchscript`     | `yolo11n.torchscript`     | ✅        | `imgsz`, `optimize`, `batch`                                         |
| [ONNX](https://docs.ultralytics.com/integrations/onnx)                   | `onnx`            | `yolo11n.onnx`            | ✅        | `imgsz`, `half`, `dynamic`, `simplify`, `opset`, `batch`             |
| [OpenVINO](https://docs.ultralytics.com/integrations/openvino)           | `openvino`        | `yolo11n_openvino_model/` | ✅        | `imgsz`, `half`, `dynamic`, `int8`, `batch`                          |
| [TensorRT](https://docs.ultralytics.com/integrations/tensorrt)           | `engine`          | `yolo11n.engine`          | ✅        | `imgsz`, `half`, `dynamic`, `simplify`, `workspace`, `int8`, `batch` |
| [CoreML](https://docs.ultralytics.com/integrations/coreml)               | `coreml`          | `yolo11n.mlpackage`       | ✅        | `imgsz`, `half`, `int8`, `nms`, `batch`                              |
| [TF SavedModel](https://docs.ultralytics.com/integrations/tf-savedmodel) | `saved_model`     | `yolo11n_saved_model/`    | ✅        | `imgsz`, `keras`, `int8`, `batch`                                    |
| [TF GraphDef](https://docs.ultralytics.com/integrations/tf-graphdef)     | `pb`              | `yolo11n.pb`              | ❌        | `imgsz`, `batch`                                                     |
| [TF Lite](https://docs.ultralytics.com/integrations/tflite)              | `tflite`          | `yolo11n.tflite`          | ✅        | `imgsz`, `half`, `int8`, `batch`                                     |
| [TF Edge TPU](https://docs.ultralytics.com/integrations/edge-tpu)        | `edgetpu`         | `yolo11n_edgetpu.tflite`  | ✅        | `imgsz`                                                              |
| [TF.js](https://docs.ultralytics.com/integrations/tfjs)                  | `tfjs`            | `yolo11n_web_model/`      | ✅        | `imgsz`, `half`, `int8`, `batch`                                     |
| [PaddlePaddle](https://docs.ultralytics.com/integrations/paddlepaddle)   | `paddle`          | `yolo11n_paddle_model/`   | ✅        | `imgsz`, `batch`                                                     |
| [MNN](https://docs.ultralytics.com/integrations/mnn)                     | `mnn`             | `yolo11n.mnn`             | ✅        | `imgsz`, `batch`, `int8`, `half`                                     |
| [NCNN](https://docs.ultralytics.com/integrations/ncnn)                   | `ncnn`            | `yolo11n_ncnn_model/`     | ✅        | `imgsz`, `half`, `batch`                                             |
| [IMX500](https://docs.ultralytics.com/integrations/sony-imx500)          | `imx`             | `yolov8n_imx_model/`      | ✅        | `imgsz`, `int8`                                                      |
| [RKNN](https://docs.ultralytics.com/integrations/rockchip-rknn)          | `rknn`            | `yolo11n_rknn_model/`     | ✅        | `imgsz`, `batch`, `name`                                             |

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO(f"{model.trainer.save_dir}/weights/best.pt")  # load a fine-tuned model

# Export the model
model.export(format="onnx")